### Importaciones necesarias para la lectura de imagenes, mostrar imagenes y realizar los calculos solicitados
recordar tener intaladas las bibliotecas correspondientes antes de ejecutar los programas
sino lo tiene, abrir cmd y ejecutar los siguientes comandos:
pip install matplotlib
pip install scikit-image
pip install opencv-python

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage import  filters

### Lectura y transformacion de imagenes a escala de grises

In [ ]:
def leer_imagen(nombreimg):
  img=cv2.imread(nombreimg)
  gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  finish_img=gray_img.astype(np.float32)/255.0
  return finish_img
img1=leer_imagen("7.1.01.tiff")
img2=leer_imagen("7.1.09.tiff")
img3=leer_imagen("boat.512.tiff")
img4=leer_imagen("gitana.jpg")

# Mostrar las imagenes que se utlizaran para las pruebas
plt.figure(figsize=(16, 4))

plt.subplot(1, 4, 1)
plt.imshow(img1, cmap='gray')
plt.title('Imagen 1')
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(img2, cmap='gray')
plt.title('Imagen 2')
plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(img3, cmap='gray')
plt.title('Imagen 3')
plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(img4, cmap='gray')
plt.title('Imagen 4')
plt.axis('off')

plt.tight_layout()
plt.show()


### Problematica y objetivo
Al realizar difuminacion isotropica se pierde informacion de la imagen ya que difumnina la imagen en todas las direcciones por igual, por lo que no reconoce el ruido que realmente se quiere eliminar y debido a esto se pierde informacion de la imagen y se obtiene una imagen difuminada. como por ejemplo el filtro gaussiano. por lo que se busca implementar la difusion anisotropica, la cual realiza un suavizado inteligente distinguiendo si esta en un borde o no, asi evitando la perdida de informacion de la imagen y manteniendo los bordes.

In [ ]:
image_gauss = filters.gaussian(img1, sigma=20)
plt.figure(figsize=(16, 4))
plt.subplot(1, 4, 1)
plt.imshow(image_gauss, cmap='gray')
plt.title('Imagen 1 con filtro gaussiano')
plt.axis('off')
plt.tight_layout()
plt.show()

### Implementacion de estimadores de coeficientes de difusion

1. **Perona-Malik tipo 1:**

$$
g(|x|) = \exp\left(-\frac{|x|^2}{K^2}\right)
$$

2. **Perona-Malik tipo 2:**

$$
g(|x|) = \frac{1}{1 + \frac{|x|^2}{K^2}}
$$

3. **Charbonnier:**

$$
g(|x|) = \frac{1}{\sqrt{1 + \frac{|x|^2}{K^2}}}
$$

la implementaciones de estas seran la clave para controlar la difuminacion de los bordes de las imagenes, los cuales se encargaran de mantenerlos la mayor cantidad de veces posibles


In [ ]:
def pm1(grad,k):
    return np.exp(-(grad/ k)**2)
def pm2(grad, K):
    return 1.0 / (1.0 + (grad / K)**2)

def ch(grad, K):
    return 1.0 / np.sqrt(1.0 + (grad / K)**2)

### Implementacion de busqueda de K basado en el estimador de bordes de canny

In [ ]:
def estimar_k(imagen_gris, percentil=90):
    sobel_x = cv2.Sobel(imagen_gris, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(imagen_gris, cv2.CV_64F, 0, 1, ksize=3)
    
    magnitud_gradiente = np.sqrt(sobel_x**2 + sobel_y**2)
    
    valores_gradiente = magnitud_gradiente.flatten()
    
    k_estimado = np.percentile(valores_gradiente, percentil)
    
    return k_estimado

k1= estimar_k(img1)
k2= estimar_k(img2)
k3= estimar_k(img3)
k4= estimar_k(img4)

### Implementacion del filtro de la difusion anisotropica. 
buscando calcular los gradientes de cada pixel en las direcciones N,S,E,O con tal de identificar cuanto difuminar dependiendo si se encuentra en un borde o no. luego calcular el coeficiente de c con cada gradiente y obtener dicho valor, luego este se multiplica por el valor lambda y se le suma a la imagen.
Aca estamos trabajando en el paso del tiempo por lo que se actualiza constantemente la imagen anterior en base al cambio permitido por el coeficiente multiplicado por el valor lambda que estima cuanto se difumina por ciclo la imagen. un mayor valor de lambda hace que la convergencia sea mas rapida.

In [ ]:
def dfan(img, it, K, lambda_val, diffusion_func):

    # Copia de la imagen para no modificar la original
    diffused_image = img.copy()

    for _ in range(it):
        # Calcular gradientes direccionales por cada direccion de los pixeles para asi verificar si estoy o no en un borde y calcular la difusión
        grad_n = np.roll(diffused_image, -1, axis=0) - diffused_image
        grad_s = np.roll(diffused_image, 1, axis=0) - diffused_image
        grad_e = np.roll(diffused_image, -1, axis=1) - diffused_image
        grad_w = np.roll(diffused_image, 1, axis=1) - diffused_image

        # Calcular coeficientes de difusión usando la función elegida
        c_n = diffusion_func(np.abs(grad_n), K)
        c_s = diffusion_func(np.abs(grad_s), K)
        c_e = diffusion_func(np.abs(grad_e), K)
        c_w = diffusion_func(np.abs(grad_w), K)

        # Aplicar la fórmula de actualización
        c = (c_n * grad_n + c_s * grad_s + c_e * grad_e + c_w * grad_w)
        diffused_image += lambda_val * c

    return diffused_image

# Pruebas por tipo de funcion de difusion anisotroprica
se probaran diversos valores de λ, k e it. Para verificar como influyen estos valores en el resultado de las imagenes
se realizaran 3 pruebas distintas por funcion de difusion para ver si los resultados son similares o un metodo es mejor que otro

### Perona-malik 1

In [ ]:
# --- Parámetros ---
ITERATIONS = 30
LAMBDA = 0.25

# --- Agrupar imágenes para procesarlas en bucle ---
original_images = [img1, img2, img3, img4]
image_titles = ['Imagen 1', 'Imagen 2', 'Imagen 3', 'Imagen 4']

k_values = [k1, k2, k3, k4]

filtered_images = []
for i in range(len(original_images)):
    # Se usa original_images[i] y su k_values[i] correspondiente
    img_filtrada = dfan(original_images[i], ITERATIONS, k_values[i], LAMBDA, pm1)
    filtered_images.append(img_filtrada)


plt.figure(figsize=(20, 5)) 

for i in range(len(filtered_images)):
    # Crear un subplot en una cuadrícula de 1 fila y 4 columnas
    plt.subplot(1, 4, i + 1)
    
    # Mostrar la imagen filtrada
    plt.imshow(filtered_images[i], cmap='gray')
    
    title_string = f'{image_titles[i]}\nλ={LAMBDA}, K={k_values[i]:.2f}, it={ITERATIONS}'
    plt.title(title_string)
    
    # Ocultar los ejes
    plt.axis('off')

# Ajustar y mostrar el gráfico final
plt.tight_layout()
plt.show()

In [ ]:

ITERATIONS_ALT = 50
LAMBDA_ALT = 0.25
K_MANUAL = 20  # Valor de K elegido manualmente, diferente a los k calculados

filtered_images_alt = []
for img in original_images:
    img_filtrada_alt = dfan(img, ITERATIONS_ALT, K_MANUAL, LAMBDA_ALT, pm1)
    filtered_images_alt.append(img_filtrada_alt)

plt.figure(figsize=(20, 5))
for i in range(len(filtered_images_alt)):
    plt.subplot(1, 4, i + 1)
    plt.imshow(filtered_images_alt[i], cmap='gray')
    plt.title(f'{image_titles[i]}\nλ={LAMBDA_ALT}, K={K_MANUAL}, it={ITERATIONS_ALT}')
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Ejemplo adicional: comparación de diferentes valores de λ, K e iteraciones para la imagen 1

valores_lambda = [0.15, 0.25, 0.35]
valores_k = [k1, 2*k1, 0.5*k1]
valores_iter = [10, 30, 60]

plt.figure(figsize=(15, 10))
for idx, (lmbd, k, it) in enumerate(zip(valores_lambda, valores_k, valores_iter)):
    img_filtrada = dfan(img1, it, k, lmbd, pm1)
    plt.subplot(2, 3, idx+1)
    plt.imshow(img_filtrada, cmap='gray')
    plt.title(f'λ={lmbd}, K={k:.2f}, it={it}')
    plt.axis('off')

    img_filtrada = dfan(img1, it, k, lmbd, pm2)
    plt.subplot(2, 3, idx+4)
    plt.imshow(img_filtrada, cmap='gray')
    plt.title(f'PM2: λ={lmbd}, K={k:.2f}, it={it}')
    plt.axis('off')

plt.tight_layout()
plt.show()

### PERONA MALIK 2

In [ ]:
# --- Parámetros ---
ITERATIONS = 30
LAMBDA = 0.25


original_images = [img1, img2, img3, img4]
image_titles = ['Imagen 1', 'Imagen 2', 'Imagen 3', 'Imagen 4']

k_values = [k1, k2, k3, k4]


filtered_images = []
for i in range(len(original_images)):
    # Se usa original_images[i] y su k_values[i] correspondiente
    img_filtrada = dfan(original_images[i], ITERATIONS, k_values[i], LAMBDA, pm2)
    filtered_images.append(img_filtrada)


plt.figure(figsize=(20, 5)) 

print("Generando gráfico...")
for i in range(len(filtered_images)):
    plt.subplot(1, 4, i + 1)
    
    # Mostrar la imagen filtrada
    plt.imshow(filtered_images[i], cmap='gray')
    

    title_string = f'{image_titles[i]}\nλ={LAMBDA}, K={k_values[i]:.2f}, it={ITERATIONS}'
    plt.title(title_string)
    
    # Ocultar los ejes
    plt.axis('off')

# Ajustar y mostrar el gráfico final
plt.tight_layout()
plt.show()

In [ ]:
# Ejemplo 1: Parámetros personalizados para pm2
ITER_1 = 20
LAMBDA_1 = 0.18
K_1 = 13

filtered_images_pm2_1 = []
for i in range(len(original_images)):
    img_filtrada = dfan(original_images[i], ITER_1, K_1, LAMBDA_1, pm2)
    filtered_images_pm2_1.append(img_filtrada)

plt.figure(figsize=(20, 5))
for i in range(len(filtered_images_pm2_1)):
    plt.subplot(1, 4, i + 1)
    plt.imshow(filtered_images_pm2_1[i], cmap='gray')
    plt.title(f'{image_titles[i]}\nλ={LAMBDA_1}, K={K_1:.2f}, it={ITER_1}')
    plt.axis('off')
plt.tight_layout()
plt.show()

# Ejemplo 2: Otro conjunto de parámetros para pm2
ITER_2 = 40
LAMBDA_2 = 0.25
K_2 = 8

filtered_images_pm2_2 = []
for i in range(len(original_images)):
    img_filtrada = dfan(original_images[i], ITER_2, K_2, LAMBDA_2, pm2)
    filtered_images_pm2_2.append(img_filtrada)

plt.figure(figsize=(20, 5))
for i in range(len(filtered_images_pm2_2)):
    plt.subplot(1, 4, i + 1)
    plt.imshow(filtered_images_pm2_2[i], cmap='gray')
    plt.title(f'{image_titles[i]}\nλ={LAMBDA_2}, K={K_2:.2f}, it={ITER_2}')
    plt.axis('off')
plt.tight_layout()
plt.show()

### Charbonnier

In [ ]:
# --- Parámetros ---
ITERATIONS = 30
LAMBDA = 0.25


original_images = [img1, img2, img3, img4]
image_titles = ['Imagen 1', 'Imagen 2', 'Imagen 3', 'Imagen 4']


k_values = [k1, k2, k3, k4]

filtered_images = []
for i in range(len(original_images)):

    img_filtrada = dfan(original_images[i], ITERATIONS, k_values[i], LAMBDA, ch)
    filtered_images.append(img_filtrada)


plt.figure(figsize=(20, 5)) 

for i in range(len(filtered_images)):

    plt.subplot(1, 4, i + 1)
    
    # Mostrar la imagen filtrada
    plt.imshow(filtered_images[i], cmap='gray')

    title_string = f'{image_titles[i]}\nλ={LAMBDA}, K={k_values[i]:.2f}, it={ITERATIONS}'
    plt.title(title_string)
    
    # Ocultar los ejes
    plt.axis('off')

# Ajustar y mostrar el gráfico final
plt.tight_layout()
plt.show()

In [ ]:
import random
from skimage.util import random_noise

# Ejemplo 1: λ=0.20, K=10, iteraciones=25 usando pm1
LAMBDA_EX1 = 0.20
K_EX1 = 10
# Agregar ruido sal y pimienta a las imágenes originales
noisy_images_sp = [random_noise(img, mode='s&p', amount=0.05) for img in original_images]

# Agregar ruido gaussiano a las imágenes originales
noisy_images_gauss = [random_noise(img, mode='gaussian', var=0.01) for img in original_images]

# Mostrar ejemplos de imágenes con ruido
plt.figure(figsize=(20, 5))
for i in range(len(noisy_images_sp)):
    plt.subplot(2, 4, i + 1)
    plt.imshow(noisy_images_sp[i], cmap='gray')
    plt.title(f'{image_titles[i]}\nSal y pimienta')
    plt.axis('off')
for i in range(len(noisy_images_gauss)):
    plt.subplot(2, 4, i + 5)
    plt.imshow(noisy_images_gauss[i], cmap='gray')
    plt.title(f'{image_titles[i]}\nGaussiano')
    plt.axis('off')
plt.tight_layout()
plt.show()
ITER_EX1 = 25

filtered_images_ex1 = []
for img in original_images:
    img_filtrada = dfan(img, ITER_EX1, K_EX1, LAMBDA_EX1, ch)
    filtered_images_ex1.append(img_filtrada)

plt.figure(figsize=(20, 5))
for i in range(len(filtered_images_ex1)):
    plt.subplot(1, 4, i + 1)
    plt.imshow(filtered_images_ex1[i], cmap='gray')
    plt.title(f'{image_titles[i]}\nλ={LAMBDA_EX1}, K={K_EX1}, it={ITER_EX1}')
    plt.axis('off')
plt.tight_layout()
plt.show()

# Ejemplo 2: λ=0.12, K=5, iteraciones=40 usando pm2
LAMBDA_EX2 = 0.12
K_EX2 = 5
ITER_EX2 = 40

filtered_images_ex2 = []
for img in original_images:
    img_filtrada = dfan(img, ITER_EX2, K_EX2, LAMBDA_EX2, ch)
    filtered_images_ex2.append(img_filtrada)

plt.figure(figsize=(20, 5))
for i in range(len(filtered_images_ex2)):
    plt.subplot(1, 4, i + 1)
    plt.imshow(filtered_images_ex2[i], cmap='gray')
    plt.title(f'{image_titles[i]}\nλ={LAMBDA_EX2}, K={K_EX2}, it={ITER_EX2}')
    plt.axis('off')
plt.tight_layout()
plt.show()